# Pseudocode
3/28/22
Generate the following:
1. Train xor
2. Test xor
3. rxor 45 degrees

Traing RF on train xor

Push train xor (troubleshooting), test xor, test rxor through the RF using .apply() -> now we have arrays with leaf nodes that each sample ends up in

Predict on test xor and rxor to get class labels for each sample -> array with 0's and 1's (y_pred_[r]xor)

For test xor and rxor:
    Get and iterate through unique polytopes (leaf nodes) -> get their idx's -> use y_pred_[r]xor[idx] to count number of 0's and 1's at each unique polytope
    
Might need to concat all unique idx's in xor and rxor sinec some only appear in one so we'd want the following dict to contain the polyope anyway ie [23: 0, 0] (example if xor didnt have poly 23 but rxor did)

### Next steps:
Instead of printing, store these counts of 0's and 1's in a dictionary with key = polytope and values = [# of 0's, # of 1's]

For xor and rxor seperately, calculate % of samples at each polytope the belong to each label ie. [5: 10, 10] -> [5: 0.5, 0.5]

Calculate l2 distance for each polytope between xor and rxor with above dictionary containing key = polytope, values = [%samples this polytope label0, [%samples this polytope label1]

In [1]:
# import 
import numpy as np
import random
from proglearn.sims import generate_gaussian_parity
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sc
import sklearn.ensemble
from sklearn import metrics 
import math
from scipy.stats import ttest_ind

In [63]:
# number of samples
n_samples = 100

# generate xor 
X_train, y_train = generate_gaussian_parity(n_samples, angle_params=0)
# X_test, y_test = generate_gaussian_parity(100, angle_params=0)

# generate rxor, 45 degrees
X_test_rxor, y_test_rxor = generate_gaussian_parity(n_samples, angle_params=np.pi/4)

In [64]:
clf = sklearn.ensemble.RandomForestClassifier(n_estimators=1)

In [65]:
# fit the model using the train data 
clf.fit(X_train, y_train)

# # predict using the test data
# y_pred = clf.predict(X_test)

RandomForestClassifier(n_estimators=1)

In [66]:
# index of each array that each sample ends up in
xor_leaves = clf.apply(X_train)
rxor_leaves = clf.apply(X_test_rxor)
# train_leaves = clf.apply(X_train)

In [67]:
# predicted test xor labels
y_pred_xor = clf.predict(X_train)

In [68]:
# predicted rxor labels
y_pred_rxor = clf.predict(X_test_rxor)

In [69]:
for uni in np.unique(xor_leaves):
    # idx of each unique polytope 
    poly_to_test = np.where(xor_leaves == uni)[0]
    num_0 = 0
    num_1 = 0

    for i in poly_to_test:
        if y_train[i] == 0: 
            num_0+=1
        else:
            num_1+=1
            
    print('node', uni, ':', num_0, num_1)

node 4 : 0 18
node 5 : 8 0
node 6 : 2 8
node 9 : 21 0
node 11 : 0 1
node 12 : 3 0
node 13 : 0 9
node 15 : 8 5
node 17 : 12 0
node 18 : 0 5


In [70]:
for uni in np.unique(rxor_leaves):
    # idx of each unique polytope
    poly_to_test = np.where(rxor_leaves == uni)[0]
    num_0 = 0
    num_1 = 0

    for i in poly_to_test:
        if y_test_rxor[i] == 0: 
            num_0+=1
        else:
            num_1+=1
            
    print('node', uni, ':', num_0, num_1)

node 4 : 11 2
node 5 : 5 0
node 6 : 2 19
node 9 : 9 8
node 11 : 2 1
node 12 : 10 0
node 13 : 3 0
node 15 : 3 6
node 17 : 0 12
node 18 : 0 7


In [71]:
np.unique(xor_leaves)

array([ 4,  5,  6,  9, 11, 12, 13, 15, 17, 18], dtype=int64)

In [72]:
np.unique(rxor_leaves)

array([ 4,  5,  6,  9, 11, 12, 13, 15, 17, 18], dtype=int64)

## Next we need to modify our functions so that they return %'s, then use them to calc L2 distance

In [77]:
perc_labels_at_poly_xor = {}
for uni in np.unique(xor_leaves):
    # idx of each unique polytope 
    poly_to_test = np.where(xor_leaves == uni)[0]
    num_0 = 0
    num_1 = 0
    
    # sum the number of each label at each poly/leaf
    for i in poly_to_test:
        if y_train[i] == 0: 
            num_0+=1
        else:
            num_1+=1
            
    # calc % of each label at each polytope/leaf
    total_samples_at_poly = num_0 + num_1
    perc_0 = num_0 / total_samples_at_poly
    perc_1 = num_1 / total_samples_at_poly
    
    perc_labels_at_poly_xor[uni] = [perc_0, perc_1]

In [80]:
perc_labels_at_poly_xor

{4: [0.0, 1.0],
 5: [1.0, 0.0],
 6: [0.2, 0.8],
 9: [1.0, 0.0],
 11: [0.0, 1.0],
 12: [1.0, 0.0],
 13: [0.0, 1.0],
 15: [0.6153846153846154, 0.38461538461538464],
 17: [1.0, 0.0],
 18: [0.0, 1.0]}

In [81]:
perc_labels_at_poly_rxor = {}
for uni in np.unique(rxor_leaves):
    # idx of each unique polytope 
    poly_to_test = np.where(rxor_leaves == uni)[0]
    num_0 = 0
    num_1 = 0
    
    # sum the number of each label at each poly/leaf
    for i in poly_to_test:
        if y_test_rxor[i] == 0: 
            num_0+=1
        else:
            num_1+=1
            
    # calc % of each label at each polytope/leaf
    total_samples_at_poly = num_0 + num_1
    perc_0 = num_0 / total_samples_at_poly
    perc_1 = num_1 / total_samples_at_poly
    
    perc_labels_at_poly_rxor[uni] = [perc_0, perc_1]

In [82]:
perc_labels_at_poly_rxor

{4: [0.8461538461538461, 0.15384615384615385],
 5: [1.0, 0.0],
 6: [0.09523809523809523, 0.9047619047619048],
 9: [0.5294117647058824, 0.47058823529411764],
 11: [0.6666666666666666, 0.3333333333333333],
 12: [1.0, 0.0],
 13: [1.0, 0.0],
 15: [0.3333333333333333, 0.6666666666666666],
 17: [0.0, 1.0],
 18: [0.0, 1.0]}

In [85]:
# function to do above but with any input (xor or rxor)
def percentLabels(leaves, true_labels):
    # dict to hold the %'s 
    # polytope as key, value = [% samples at this polytope with label 0, % samples at this polytope with label 1]
    perc_labels_at_poly = {}
    
    for uni in np.unique(leaves):
        # idx of each unique polytope 
        poly_to_test = np.where(leaves == uni)[0]
        num_0 = 0
        num_1 = 0

        # sum the number of each label at each poly/leaf
        for i in poly_to_test:
            if true_labels[i] == 0: 
                num_0+=1
            else:
                num_1+=1

        # calc % of each label at each polytope/leaf
        total_samples_at_poly = num_0 + num_1
        perc_0 = num_0 / total_samples_at_poly
        perc_1 = num_1 / total_samples_at_poly

        perc_labels_at_poly[uni] = [perc_0, perc_1]
        
    return perc_labels_at_poly
    

In [88]:
perc_labels_at_poly_xor_2 = percentLabels(xor_leaves, y_train)
perc_labels_at_poly_rxor_2 = percentLabels(rxor_leaves, y_test_rxor)

In [89]:
# make sure the function is working correctly 
perc_labels_at_poly_xor_2 == perc_labels_at_poly_xor, perc_labels_at_poly_rxor_2 == perc_labels_at_poly_rxor

(True, True)

In [91]:
# l2 distance
from scipy.spatial import distance
distance.euclidean(perc_labels_at_poly_xor[4], perc_labels_at_poly_rxor[4])

1.1966422450849266

In [95]:
# function to calculate l2 distance between each rxor and xor polytope
def l2_polytopes(xor_dict, rxor_dict):
    # polytopes as keys, l2 distances as value
    l2_poly_dict = {}
    
    # calc euc distance between each poly and store them in our dict
    for key in xor_dict.keys():
        l2_poly_dict[key] = distance.euclidean(xor_dict[key], rxor_dict[key])
        
    return l2_poly_dict

In [96]:
l2_dists = l2_polytopes(perc_labels_at_poly_xor_2, perc_labels_at_poly_rxor_2)
l2_dists

{4: 1.1966422450849266,
 5: 0.0,
 6: 0.14815570653432422,
 9: 0.6655122646461624,
 11: 0.9428090415820634,
 12: 0.0,
 13: 1.4142135623730951,
 15: 0.39888074836164217,
 17: 1.4142135623730951,
 18: 0.0}